### Importing libraries

In [182]:
import pandas as pd

### Loading data

##### Processed dataset with pre-selecter features (by Random Forest)

In [183]:
# data_train = pd.read_csv('data/processed/train_meaningfull.csv')
# data_test = pd.read_csv('data/processed/test_meaningfull.csv')

##### Raw dataset with (>500 features)

In [184]:
data_train = pd.read_csv('data/raw/train.csv')
data_test = pd.read_csv('data/raw/test.csv')

##### Processing data

In [185]:
X_train = data_train.drop('Activity', axis='columns')
y_train = data_train['Activity']
y_train_encoded = pd.get_dummies(y_train)

In [186]:
X_test = data_test.drop('Activity', axis='columns')
y_test = data_test['Activity']
y_test_encoded = pd.get_dummies(y_test)

### Build model

In [187]:
from tensorflow import keras
from keras import layers

model = keras.Sequential(
  [
    layers.Input(X_train.shape[-1]),
    layers.Dense(6, activation="sigmoid"),
  ]
)
model.summary()

Model: "sequential_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_91 (Dense)            (None, 6)                 3378      
                                                                 
Total params: 3,378
Trainable params: 3,378
Non-trainable params: 0
_________________________________________________________________


In [188]:
optimizer = keras.optimizers.Adam()

model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics='accuracy')

In [189]:
model.fit(
  X_train,
  y_train_encoded,
  batch_size=400,
  epochs=30,
  verbose=2,
  validation_split=0.2
)

Epoch 1/30


15/15 - 0s - loss: 1.8707 - accuracy: 0.2882 - val_loss: 1.4118 - val_accuracy: 0.3426 - 462ms/epoch - 31ms/step
Epoch 2/30
15/15 - 0s - loss: 1.2637 - accuracy: 0.4319 - val_loss: 1.1676 - val_accuracy: 0.6417 - 127ms/epoch - 8ms/step
Epoch 3/30
15/15 - 0s - loss: 1.0021 - accuracy: 0.6291 - val_loss: 0.9747 - val_accuracy: 0.6914 - 122ms/epoch - 8ms/step
Epoch 4/30
15/15 - 0s - loss: 0.8502 - accuracy: 0.7313 - val_loss: 0.7945 - val_accuracy: 0.7573 - 118ms/epoch - 8ms/step
Epoch 5/30
15/15 - 0s - loss: 0.7431 - accuracy: 0.7784 - val_loss: 0.7175 - val_accuracy: 0.7736 - 106ms/epoch - 7ms/step
Epoch 6/30
15/15 - 0s - loss: 0.6617 - accuracy: 0.8223 - val_loss: 0.6289 - val_accuracy: 0.8083 - 87ms/epoch - 6ms/step
Epoch 7/30
15/15 - 0s - loss: 0.5984 - accuracy: 0.8490 - val_loss: 0.5665 - val_accuracy: 0.8559 - 85ms/epoch - 6ms/step
Epoch 8/30
15/15 - 0s - loss: 0.5485 - accuracy: 0.8623 - val_loss: 0.5072 - val_accuracy: 0.9028 - 92ms/epoch - 6ms/step
Epoch 9/30
15/15 - 0s - loss:

In [190]:
score = model.evaluate(X_test, y_test_encoded)[1]

score

93/93 [==============================] - 0s 3ms/step - loss: 0.2846 - accuracy: 0.9250


0.9250084757804871